In [320]:
import numpy as np
import os 
import pandas as pd

Valores importantes

K = numero de clusters
p = numero de matrizes de dissimilaridade
n = numero de exemplos


Dimensionalidades:
G = p x K  
U = N x K  
A = p x K
D = p x n x n
D' = n x k 

term_1  = K x n
term_2  = x 

In [321]:
mode = 'no_class'
metrics = ['euclidean','cityblock','chebyshev']

input_path = [
    os.path.join( '../data/diss_matrices/{}/diss_matrix_{}_tests.csv'.format(mode,metric) ) for metric in metrics
    ]
print(input_path)
D = [np.expand_dims(pd.read_csv(filename).to_numpy(),axis=0) for filename in input_path]


D = np.concatenate( D,axis=0)
D = D[:,:,1:]


['../data/diss_matrices/no_class/diss_matrix_euclidean_tests.csv', '../data/diss_matrices/no_class/diss_matrix_cityblock_tests.csv', '../data/diss_matrices/no_class/diss_matrix_chebyshev_tests.csv']


In [322]:
def normalize_data(D):
    print(D.shape)
    l = ([p.sum(axis=1).argmin() for p in list(D)])
    print(np.asarray(l).shape)
    k  =[ np.sum(D[i][l[i]]) for i in range(len(l)) ]
    k = np.asarray(k)[...,np.newaxis,np.newaxis]
    D = np.divide(D,k)
    return D
    
    
l = normalize_data(D)
print(l)

(3, 499, 499)
(3,)
[[[0.         0.00305977 0.0018649  ... 0.00304117 0.00307501 0.00277842]
  [0.00305977 0.         0.00310133 ... 0.00088202 0.00187733 0.0010486 ]
  [0.0018649  0.00310133 0.         ... 0.0028556  0.00260393 0.00272394]
  ...
  [0.00304117 0.00088202 0.0028556  ... 0.         0.00146558 0.00083038]
  [0.00307501 0.00187733 0.00260393 ... 0.00146558 0.         0.00126269]
  [0.00277842 0.0010486  0.00272394 ... 0.00083038 0.00126269 0.        ]]

 [[0.         0.00306299 0.00207389 ... 0.00309501 0.00276288 0.00248635]
  [0.00306299 0.         0.00320107 ... 0.00096853 0.00205168 0.00123643]
  [0.00207389 0.00320107 0.         ... 0.00263083 0.00200436 0.00261413]
  ...
  [0.00309501 0.00096853 0.00263083 ... 0.         0.00151246 0.00076311]
  [0.00276288 0.00205168 0.00200436 ... 0.00151246 0.         0.00130132]
  [0.00248635 0.00123643 0.00261413 ... 0.00076311 0.00130132 0.        ]]

 [[0.         0.00346448 0.0018427  ... 0.00348327 0.00357721 0.00329163]
  [

In [323]:
def compute_J(G,U,D,A,m=2):
    Ut = np.power(U,m)
    A = np.einsum('pk,pkn->kn',A,D[:,G,:]) 
    return np.einsum('kn,kn->',A,Ut)


In [358]:
def compute_U_fuzzy_part(G, U, D, A, m):
    term_1  = np.einsum('pk,pkn->kn',A,D[G,...])
    term_2  = np.einsum('pk,pkn->n',A,D[G,...] )  

    l = np.divide(term_1, term_2)

    k = np.power( l, (1/(m-1)))
    k[np.where(k == 0)] = 1
    
    Ut = np.divide(1,k)
    return Ut



In [359]:
def search_best_relev_vec(G,U,D,A,m):

    Ut = np.power(U,m)
    l = np.einsum('kn,pkn->pk',Ut,D[:,G,:])

    a = np.prod(l,axis=0)
    a = np.power(a,(1/(D.shape[0])))
    al = np.divide(a,l)
    
    return al

In [331]:
def search_best_medoids(G,U,D,A,m):

    all = []
    for p in range (D.shape[0]):
        al = []
        for k in range(U.shape[0]):
            h = np.sum(np.power(U[k],m)*D[p],axis=1).argmin()
            al.append(h)
        all.append(al)
    
    all = np.asarray(all)
    return all


In [346]:
def mv_algorithm(D, K, m, T, e):

    """
    
    """

    ########## CONSTRAINTS ##############
     
    assert D.shape[1] == D.shape[2]
    assert K > 1 and K < D.shape[1]
    assert e > 0 
    assert m > 1

    ######### INITIALIZATION #############

    p = D.shape[0]
    n = D.shape[1]
  

    G          = np.random.randint(low=0,high=n,size=(p,K))
    A          = np.ones(shape=(p,K))
    U          = compute_U_fuzzy_part(G,None,D,A,m)

    
   
    cur_u  = compute_J(G,U,D,A,m)
    


    last_u = 0

    ########## TRAINING LOOP ###############
    for t in range(T):
        print("G shape before search ({}) = {}".format(t,G.shape))

        G = search_best_medoids(G,U,D,A,m)
        print("G shape after search ({}) = {}".format(t,G.shape))
        #print(A.shape)
        print("D shape before search ({}) = {}".format(t,D.shape))
        A = search_best_relev_vec(G,U,D,A,m)
        print("D shape after search ({}) = {}".format(t,D.shape))
        #print(A.shape)

        U          = compute_U_fuzzy_part(G,U,D,A,m)
        


        #print(cur_u)
        
        last_u = cur_u
        cur_u = compute_J(G,U,D,A,m)
        #print(cur_u-last_u)
        if np.absolute(cur_u-last_u) < e:
            print('saiu com {}'.format(t))
            break








In [328]:

print(D.shape)
K = 5
T = 10
e = 0.1
m = 3

(3, 499, 499)


In [360]:
mv_algorithm(D, K, m, T, e)

ValueError: cannot select an axis to squeeze out which has size not equal to one